<a href="https://colab.research.google.com/github/ananddiv/datascience/blob/main/Gridsearch_and_K_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold, train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

Read the data set and display the first few rows

In [3]:
df = pd.read_csv('apple_customers_transformed.csv')
df.head()

,gender,income,kids,is_management_,iphone,macbook_owner_,airpods,age,black,hispanic,...,cooking,dancing,exercise,instruments,languages,podcasts,reading,travel,tv/movies,videogames
0,1,148523,1.0,0,1,2,1,32,0,1,...,0,0,1,0,0,0,0,0,0,0
1,1,78967,4.0,1,1,2,1,31,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,32646,1.0,1,1,1,1,44,1,0,...,0,0,1,0,0,0,0,0,0,0
3,1,41893,1.0,0,1,2,1,43,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,35485,3.0,1,1,1,1,41,0,0,...,0,0,1,0,0,0,0,0,0,0


Understand the dataset using info() and describe()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               948 non-null    int64  
 1   income               948 non-null    int64  
 2   kids                 948 non-null    float64
 3   is_management_       948 non-null    int64  
 4   iphone               948 non-null    int64  
 5   macbook_owner_       948 non-null    int64  
 6   airpods              948 non-null    int64  
 7   age                  948 non-null    int64  
 8   black                948 non-null    int64  
 9   hispanic             948 non-null    int64  
 10  other                948 non-null    int64  
 11  white                948 non-null    int64  
 12  somewhat tech savvy  948 non-null    int64  
 13  tech savvy           948 non-null    int64  
 14  cars                 948 non-null    int64  
 15  concerts             948 non-null    int

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
gender,948.0,0.251055,0.433849,0.0,0.0,0.0,1.0,1.0
income,948.0,79649.724684,45406.141019,15848.0,50021.0,70862.0,99054.5,444601.0
kids,948.0,2.452532,1.708685,0.0,1.0,2.0,4.0,5.0
is_management_,948.0,0.436709,0.496240,0.0,0.0,0.0,1.0,1.0
iphone,948.0,0.628692,0.483410,0.0,0.0,1.0,1.0,1.0
macbook_owner_,948.0,1.604430,0.489231,1.0,1.0,2.0,2.0,2.0
airpods,948.0,0.556962,0.497007,0.0,0.0,1.0,1.0,1.0
age,948.0,45.861814,16.828375,0.0,33.0,43.0,57.0,106.0
black,948.0,0.213080,0.409700,0.0,0.0,0.0,0.0,1.0
hispanic,948.0,0.105485,0.307340,0.0,0.0,0.0,0.0,1.0


Define X and y

In [6]:
X= df.drop('airpods', axis=1)
y=df['airpods']

Perform Test/Train Split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)

Scale and transform the data

In [8]:
st=StandardScaler()

In [9]:
X_train_scaled = st.fit_transform(X_train)
X_test_scaled = st.transform(X_test)

Define the metrics function

In [10]:
def metrics(y_test, preds):
    print(accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))
    print(confusion_matrix(y_test, preds))

In [21]:
svc = SVC()
svc.fit(X_train_scaled,y_train)
svc_preds = svc.predict(X_test_scaled)
metrics(y_test,svc_preds)

0.8052631578947368
              precision    recall  f1-score   support

           0       0.76      0.79      0.78        81
           1       0.84      0.82      0.83       109

    accuracy                           0.81       190
   macro avg       0.80      0.80      0.80       190
weighted avg       0.81      0.81      0.81       190

[[64 17]
 [20 89]]


In [15]:
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [30]:
params = {
    'kernel':['rbf','linear'],
    'C':[.1,1,10,100],
    'gamma':[.001,.01,.1,1,10]

}

In [31]:
gs = GridSearchCV(estimator = SVC(),param_grid=params,scoring='accuracy')
gs.fit(X_train_scaled,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [0.001, 0.01, 0.1, 1, 10],
                         'kernel': ['rbf', 'linear']},
             scoring='accuracy')

In [32]:
gs.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [29]:
gs_preds = gs.predict(X_test_scaled)
metrics(y_test,gs_preds)

0.8157894736842105
              precision    recall  f1-score   support

           0       0.80      0.75      0.78        81
           1       0.82      0.86      0.84       109

    accuracy                           0.82       190
   macro avg       0.81      0.81      0.81       190
weighted avg       0.82      0.82      0.81       190

[[61 20]
 [15 94]]


**K-Folds Cross Validation**

In [34]:
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [35]:
X_train_scaled_kfolds = X_train_scaled

In [41]:
accuracy_collection = []
for train_index,val_index in kf.split(X_train_scaled_kfolds):
  X_train = X.iloc[train_index]
  X_val = X.iloc[val_index]

  y_train =y.iloc[train_index]
  y_test =y.iloc[val_index]

  rfc = RandomForestClassifier()
  rfc.fit(X_train,y_train)
  current_preds = rfc.predict(X_val)

  accuracy_collection.append(accuracy_score(y_test,current_preds))

In [43]:
np.mean(accuracy_collection)

0.7915475775531544

In [47]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
preds = rfc.predict(X_test)

metrics(y_test,preds)


0.8
              precision    recall  f1-score   support

           0       0.77      0.75      0.76        81
           1       0.82      0.83      0.83       109

    accuracy                           0.80       190
   macro avg       0.80      0.79      0.79       190
weighted avg       0.80      0.80      0.80       190

[[61 20]
 [18 91]]


**KFold - Simple Way**

In [ ]:
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [51]:
new_rfc = RandomForestClassifier()
#cv_accuracy = cross_val_score(new_rfc,X=X_train,y=y_train,scoring='accuracy',cv=kf)
cv_precision = cross_val_score(new_rfc,X=X_train,y=y_train,scoring='precision',cv=kf)

In [50]:
cv_accuracy

array([0.80921053, 0.81578947, 0.82236842, 0.80794702, 0.77483444])

In [52]:
cv_precision

array([0.77108434, 0.87341772, 0.84705882, 0.86904762, 0.73255814])